<h1>Generate and Improve Image Descriptions</h1>

In this notebook we prompt a model to describe our images and then we ask the model to improve descriptions. (a-few-shots prompt)

This involves using an image as a scenario of survey questions, and piping the answer to a question into another question.

In [12]:
from edsl import QuestionFreeText

q1 = QuestionFreeText(
    question_name = "describe",
    question_text = """
    I have an image of an object that I would like you to describe in detail. 
    Based on the image provided, please write a description that includes:

    1. The object's appearance, including color, shape, and material.
    2. Any notable design features or patterns.
    3. The object's functionality or potential use.
    4. The overall impression or aesthetic of the object.
    
    Here is the image: {{ image }}. Please make your description as precise and vivid as possible.
    """
)

In [13]:
from edsl import QuestionFreeText

q2 = QuestionFreeText(
    question_name = "improvements",
    question_text = """
    Evaluate the description of this image on a scale from 1 to 10.
    Explain the reasoning behind your evaluation.
    Image: {{ image }}
    Description: {{ describe.answer }}
    """
)

In [14]:
# all images sotred/cached on expectedparrot service.

IMAGE_UUIDS = {
    'cabin_luggage': ['4f41ad44-6472-4403-8e59-0fb0e5eca575', 'e30bbefa-b135-4a46-bf32-ed9437be8733', 'a56079a4-1f50-4019-9344-947101bd3e4b'],
    'packing_cubes': ['7c046f64-1209-447d-a68a-b984f3508e0b', '82ffd7a6-87dc-4365-9c9f-2e76263908be', '851dedbe-3b32-45ca-8c32-4ddc510f7327'],
    'water_bottle': ['2a81aec9-c164-4d29-9281-360547f04ac4', 'd34d3ec1-073d-45b9-baca-a0d3411e9e0c', 'ff0ab897-50b8-458e-bfa5-7550b8a48648']
}

In [15]:
%%time

from edsl import Scenario, ScenarioList, FileStore

def prefetch_images(image_uuids):
    return {
        product: {
            index: FileStore.pull(uuid) for index, uuid in enumerate(uuids)
        }
        for product, uuids in image_uuids.items()
    }

pre_fetched_images = prefetch_images(IMAGE_UUIDS)

sl = ScenarioList([
    Scenario({
        "question_name": f"{product}_{i + 1}",
        "image": pre_fetched_images[product][i]
    })
    for product in IMAGE_UUIDS
    for i in range(3)
])

sl

CPU times: user 256 ms, sys: 228 ms, total: 484 ms
Wall time: 12.3 s


,question_name,image
0,cabin_luggage_1,FileStore: self.path
1,cabin_luggage_2,FileStore: self.path
2,cabin_luggage_3,FileStore: self.path
3,packing_cubes_1,FileStore: self.path
4,packing_cubes_2,FileStore: self.path
5,packing_cubes_3,FileStore: self.path
6,water_bottle_1,FileStore: self.path
7,water_bottle_2,FileStore: self.path
8,water_bottle_3,FileStore: self.path


In [16]:
%%time

from edsl import Survey

survey = Survey(questions = [q1, q2])

CPU times: user 110 μs, sys: 1e+03 ns, total: 111 μs
Wall time: 112 μs


In [17]:
from edsl import Model

Model.services()

,service
0,anthropic
1,azure
2,bedrock
3,deep_infra
4,deepseek
5,google
6,groq
7,mistral
8,ollama
9,open_router


In [18]:
from edsl import Model, ModelList

ml = ModelList([
#   Model("gemini-2.5-flash-light", service_name = "google", temperature = 1, maxOutputTokens = 5000),
#   Model("claude-sonnet-4-20250514", service_name="anthropic", temperature = 1, max_tokens = 5000),
#   Model("claude-opus-4-1-20250805", service_name="anthropic", temperature = 1, max_tokens = 5000),
  Model("gpt-5-chat-latest", service_name = "openai", temperature = 1, max_tokens = 5000),
  Model("gpt-5", service_name = "openai", temperature = 1, max_tokens = 5000),
#   Model("gemini-2.0-flash", service_name = "google", temperature = 1, maxOutputTokens = 5000),
#   Model("gemini-2.5-flash", service_name = "google", temperature = 1, maxOutputTokens = 5000),
#   Model("gemini-1.5-flash", service_name = "google", temperature = 1, maxOutputTokens = 5000),
#   Model("gemini-2.5-pro", service_name = "google", temperature = 1, maxOutputTokens = 5000),
#   Model("gpt-4o", service_name = "openai", temperature = 0, max_tokens = 5000),
#   Model("anthropic.claude-3-5-sonnet-20240620-v1:0", service_name = "bedrock", temperature = 1, maxOutputTokens = 5000),
#   Model("anthropic.claude-3-opus-20240229-v1:0", service_name = "bedrock", temperature = 1, maxOutputTokens = 5000),
])

ml

# aws_bedrock_models = Model.available(service="bedrock")
# aws_bedrock_models


,model_name,service_name,temperature,max_tokens,top_p,frequency_penalty,presence_penalty,logprobs,top_logprobs
0,gpt-5-chat-latest,openai,1,5000,1,0,0,False,3
1,gpt-5,openai,1,5000,1,0,0,False,3


In [19]:
# import os
from dotenv import load_dotenv

load_dotenv()  # By default, loads from .env in the current directory

# Test if the API keys are loaded
# print(os.getenv("OPENAI_API_KEY"))
# print(os.getenv("GOOGLE_API_KEY"))
# print(os.getenv("EXPECTED_PARROT_API_KEY"))


True

In [20]:
%%time

results = survey.by(sl).by(ml).run()

CPU times: user 4.97 s, sys: 582 ms, total: 5.55 s
Wall time: 5.82 s


In [21]:
results.select('model', 'scenario.question_name', 'answer.*')

,model.model,scenario.question_name,answer.describe,answer.improvements
0,gpt-5-chat-latest,cabin_luggage_1,"The object in the image is a sleek and modern suitcase, designed with a minimalist yet highly functional aesthetic. 1. **Appearance (color, shape, material):** The suitcase is rectangular with slightly rounded edges, giving it a clean and polished silhouette. Its color is a metallic silver, which enhances its modern and premium look. The body appears to be made of durable aluminum or polycarbonate, giving it both strength and a reflective sheen. The smooth surface is contrasted by evenly spaced vertical ridges running from top to bottom, adding dimension and structure to the design. 2. **Notable design features or patterns:** The vertical grooves are the most distinctive design feature, serving both as a stylistic element and potentially reinforcing the strength of the body. The corners are reinforced with silver-toned protectors, secured by visible rivets, suggesting durability and long-lasting use. At the top, a built-in handle is seamlessly integrated into the structure, while the bottom corners feature smooth-rolling wheels designed with a circular pattern, enhancing mobility. The top front also displays a small metallic plate with engraved branding. 3. **Functionality or potential use:** This is a high-end suitcase designed for travel, offering robust protection for belongings while maintaining a stylish exterior. The strong material ensures resilience against impacts, while the multidirectional wheels allow for easy maneuvering through airports or city streets. The reinforced corners and handles suggest it is built to withstand frequent and demanding use. The vertical ridges likely help reduce visible wear and scratches. 4. **Overall impression or aesthetic:** The suitcase conveys an image of sophistication, strength, and modern luxury. Its design merges practicality with elegance, giving it a professional and cosmopolitan feel. The metallic silver color paired with the streamlined ridged surface creates a timeless and iconic aesthetic, making it stand out as both a functional travel tool and a fashion statement. Overall, this object exudes **premium craftsmanship, durability, and minimalist elegance**, appealing to travelers who value both utility and style.","I would evaluate the provided description of the image as a **9/10**. ### Reasoning: 1. **Accuracy of Physical Description (Color, Shape, Material):** - The suitcase is indeed rectangular with rounded corners. - It has a metallic silver color with vertical ridges—a central design feature the description captures correctly. - The material mentioned (aluminum or polycarbonate) is accurate; this type of suitcase is typically built with aluminum. ✅ Very precise. 2. **Design Features Recognition:** - The vertical grooves, reinforced corners, rivets, integrated handle, and multi-directional wheels with circular accents are all correctly noted. - The metallic branding plate at the top is also visible in the image and well described. ✅ Excellent level of detail—shows good observational accuracy. 3. **Functionality and Use:** - The description highlights durability, mobility, and practical travel use, which aligns with the visible features (roll wheels, hard-shell body, reinforced design). - The mention of ridges reducing scratches is insightful, though more speculative than explicitly visible. ✅ Strong reasoning, minor inference but plausible. 4. **Overall Aesthetic Impression:** - Words like ""sophistication,"" ""minimalist elegance,"" and ""premium craftsmanship"" align with both the real-world branding and the visual appearance of the suitcase. ✅ Well written and consistent with visual evidence. ### Why not a perfect 10? - The description leans slightly into subjective interpretation (""timeless and iconic aesthetic,"" ""cosmopolitan feel"") which, while stylish and likely true for this product, adds a bit of marketing-style flourish rather than strict visual an